In [1]:
import sys
#sys.path.remove('C:\\Users\\Nathan\\Documents\\Dropbox\\Python')
sys.path.remove('C:\\users\\nathan\\documents\\Dropbox\\Python\\jpnotebooks\\Customer Apps\\Semma Data Scripts\\ C:\\program files\\python35\\Lib\\site-packages')
sys.path.append('C:\\Users\\Nathan\\Documents\\Dropbox\\Python\\scripts\\customer_apps\\multilogger\\src\\')

In [2]:
#from multilogger import HelloMultiLogger

In [7]:
import threading
from hello.hello3 import open_hello, BadConnection
from datetime import datetime
import time
import sys
from requests.exceptions import Timeout

_gmv_keys = [
    'temperature.output', 
    'temperature.man', 
    'temperature.interlocked', 
    'temperature.pv', 
    'temperature.sp', 
    'temperature.error', 
    'temperature.mode', 
    'agitation.output', 
    'agitation.man', 
    'agitation.interlocked', 
    'agitation.pv', 
    'agitation.sp', 
    'agitation.error', 
    'agitation.mode', 
    'ph.manUp',
    'ph.outputDown',
    'ph.error',
    'ph.pv',
    'ph.sp',
    'ph.manDown',
    'ph.outputUp',
    'ph.mode',
    'ph.interlocked',
    'do.manUp',
    'do.outputDown',
    'do.error',
    'do.pv',
    'do.sp',
    'do.manDown',
    'do.outputUp',
    'do.mode',
    'do.interlocked',
    'maingas.error',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'maingas.interlocked',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'condenser.output',
    'condenser.man',
    'condenser.pv',
    'condenser.sp', 
    'condenser.error', 
    'condenser.mode',
    'pressure.error',
    'pressure.pv',
    'level.error',
    'level.pv'
]


def _check_vars(variables):
    for v in variables:
        if v not in _gmv_keys:
            raise ValueError("Invalid key: %s" % v)


def _parse_vars(pvs):
    seen = set()
    keys = []
    for k in pvs:
        if k in seen:
            raise ValueError("Duplicate key found: %r"%k)
        if k not in _gmv_keys:
            raise ValueError("Invalid key: %r"%k)
        seen.add(k)
        try:
            k1, k2 = k.split(".")
        except ValueError:  # no ".", bad key
            raise ValueError("%r is an invalid key" % k) from None
        keys.append((k1, k2))
    return tuple(keys)

class MultiLogError(Exception):
    pass

class MultiLogConnectionError(BadConnection):
    pass

class _HelloLoggerTask(threading.Thread):
    def __init__(self, ip, tag, variables, poll_interval=2, start_time=None):
        
        super().__init__(daemon=True)
        # Config
        self._ip = ip
        self._vars = variables
        self._tag = tag
        self.poll_interval = poll_interval
        
        # Runtime
        self.data = None
        self._h = open_hello(ip)
        self._h.settimeout(3)
        
        try:
            self._h.gpmv()
        except Exception as e:
            raise MultiLogConnectionError("Failed to connect to '%s'"%ip) from None

        if isinstance(start_time, datetime):
            start_time = start_time.timestamp()
        
        self._start_time = start_time
        
        self._iflag = threading.Event()
        self._fstop = threading.Event()
        
    def stop(self):
        self._fstop.set()
        
    @property
    def tag(self):
        return self._tag
    
    @property
    def vars(self):
        return self._vars
    
    def wait_for_initialized(self, timeout=None):
        if not self.is_alive():
            return
        if not self._iflag.wait(timeout):
            raise MultiLogError("Failed to initialize")
        
    def _query(self, h, start_time, recorded):
        
        # An existential question - when did the query actually occur?
        # Did it occur as soon as the request was sent, after it was
        # recieved, or halfway between the two? The most likely scenario
        # for a delayed response is that our server software is choking 
        # for some reason, so use the timestamp immediately after return
        # as the best estimate. Otherwise, the data parsing should be fast
        # enough that the error would be measurable in microseconds. 
        
        # The loop is here to retry "infinitely" until next_run (code
        # will re-run anyway), the stop flag is set, or success. 
        # This silently swallows errors but that's fine for a proof of
        # concept of this scale. 
        
        while True:
            try:
                mv = h.gpmv()
            except Exception:
                if self._fstop.is_set():
                    return
            else:
                break
                
        t = datetime.now()  
        data = [t, (t-start_time).total_seconds() / 3600]

        for k1, k2 in recorded:
            v = mv[k1][k2]
            data.append(v)
        self.data = data  # "Atomic"
        
    def run(self):
        
        # create some locals for brevity
        h = self._h
        h.settimeout(3)

        if self._start_time is None:
            self._start_time = datetime.now()
        start_time = self._start_time
        
        recorded = self._vars
        
        # Initial query so that self.data isn't junk
        # check the fstop here in case we need to bail
        # due to abort during startup. 
        
        self._query(h, start_time, recorded)
        if self._fstop.is_set():
            return 
        
        # should be impossible, but check just in case....
        if not self.data:
            raise ValueError("Failed to start: " + self.tag)
        
        self._iflag.set()
        
        while not self._fstop.is_set():
            next_run = time.time() + self.poll_interval
            
            self._query(h, start_time, recorded)
            
            left = next_run - time.time()
            if left > 0:
                self._fstop.wait(left)
        
        
class HelloMultiLogger(threading.Thread):
    def __init__(self, variables, sample_interval=10):
        
        super().__init__(daemon=True)
        self._tasks = []
        self._vars = _parse_vars(variables)
        self._sample_interval = sample_interval
        self._ufuncs = []
        self._running = False
        self._fstop = threading.Event()
        
    def add_callback(self, cb):
        if cb in self._ufuncs:
            return
        self._ufuncs.append(cb)
        
    def remove_callback(self, cb):
        try:
            self._ufuncs.remove(cb)
        except ValueError:
            pass
        
    def stop(self):
        for t in self._tasks:
            t.stop()
        for t in self._tasks:
            t.join()
        self._fstop.set()
        self.join()
        
    def add_logger(self, ip, tag, poll_interval=2, start_time=None):
        if self.is_alive():
            raise ValueError("Can't add tasks to already-running data logger.")
        task = _HelloLoggerTask(ip, tag, self._vars, poll_interval, start_time)
        self._tasks.append(task)
        return task
    
    def start(self):
        self._running = True
        for t in self._tasks:
            t.start()
        super().start()
        
    def get_var_order(self):
        # the order of data sent each iteration is constant and never changes
        order = ["Timestamp", "Elapsed Time(hr)"]
        for t in self._tasks:
            order.append(t.tag + '.TimeStamp.Last')
            order.append(t.tag + '.Elapsed Time.hr')
            for k1, k2 in t.vars:
                order.append(".".join((t.tag, k1, k2)))
        return order
        
    def _do_callback(self, ev, payload):
        for cb in self._ufuncs:
            try:
                cb(ev, payload)
            except Exception as e:
                sys.stderr.write("Error processing callback: %s\n" % str(e))
    
    def _send_update_cb(self, data):
        self._do_callback('ML_LOG_DATA', data)
        
    def _send_begin_cb(self):
        self._do_callback("ML_BEGIN_LOG", self.get_var_order())
        
    def _send_end_cb(self):
        self._do_callback("ML_END_LOG", None)
        
    def _initialize(self):
        abort = False
        msg = ""
        for t in self._tasks:
            try:
                t.wait_for_initialized(10)
            except Exception as e:
                abort = True
                msg = e.args[0]
                print("ERROR during startup: " + msg)
                break
        
        if abort:
            for t in self._tasks:
                t.stop()
        return not abort
        
    def run(self):
        
        start_time = datetime.now()
        
        if self._initialize():
            self._send_begin_cb()
            while not self._fstop.is_set():

                # I went back on forth on the best way to get the data off
                # each thread when it was time to log. The format of each data
                # object is a list of tuples, and the assignment of the instance
                # attribute is atomic. Therefore, just grabbing the data object 
                # without bothering with event flags or locks is fine, assuming CPython. 

                ts = datetime.now()
                next_run = time.time() + self._sample_interval

                data = [ts, (ts-start_time).total_seconds() / 3600]
                for t in self._tasks:
                    data.extend(t.data)
                self._send_update_cb(data)

                left = next_run - time.time()
                if left > 0:
                    self._fstop.wait(left)
                    
        self._send_end_cb()

In [8]:
def callback(event, payload):
    if event == "ML_BEGIN_LOG":
        pass
    elif event == "ML_LOG_DATA":
        pass
    elif event == "ML_END_LOG":
        pass
    else:
        raise ValueError("Unknown event: %r"%event)

def my_callback(event, payload):
    print("Got event:", event)
    if event == "ML_BEGIN_LOG":
        print(*payload)
    elif event == "ML_LOG_DATA":
        print(*payload)
    elif event == "ML_END_LOG":
        pass
    else:
        raise ValueError("Unknown event: %r"%event)
        
def csv_writer(filename):
    f = None
    def csv_writer_cb(event, payload):
        nonlocal f
        if event == "ML_BEGIN_LOG":
            f = open(filename, 'w')
            f.write(",".join(map(str, payload)))
            f.write("\n")
        elif event == "ML_LOG_DATA":
            f.write(",".join(map(str, payload)))
            f.write("\n")
        elif event == "ML_END_LOG":
            f.close()
        else:
            raise ValueError("Unknown event: %r"%event)
    
    return csv_writer_cb

In [9]:
vars_i_care_about = [
    'agitation.pv',
    'temperature.pv'
]

ml = HelloMultiLogger(vars_i_care_about, 3)

ml.add_logger("192.168.1.12", "R&D1", 1)
ml.add_logger("192.168.1.16", "R&D2", 1)

ml.add_callback(my_callback)
ml.add_callback(csv_writer("test_multilogger.csv"))

ml.start()

Got event: ML_BEGIN_LOG
Timestamp Elapsed Time(hr) R&D1.TimeStamp.Last R&D1.Elapsed Time.hr R&D1.agitation.pv R&D1.temperature.pv R&D2.TimeStamp.Last R&D2.Elapsed Time.hr R&D2.agitation.pv R&D2.temperature.pv
Got event: ML_LOG_DATA
2018-10-09 17:33:29.690337 2.737861111111111e-05 2018-10-09 17:33:29.684671 2.6636944444444445e-05 0 25.85523223876953 2018-10-09 17:33:29.680371 2.5164999999999997e-05 0 29.520050048828125
Got event: ML_LOG_DATA
2018-10-09 17:33:32.691889 0.0008611430555555556 2018-10-09 17:33:31.787469 0.0006107475000000001 0 25.853681564331055 2018-10-09 17:33:31.791678 0.0006116391666666667 0 29.5195369720459


In [10]:
ml.stop()

Got event: ML_END_LOG


In [ ]:
print("test")